# SimPy Example for IE 306.02
This example shows how a simple G/G/1 system (1 server with an infinite capacity queue, random interarrival and service times) can be modeled with a process-interaction view using the SimPy library. 

In this example customers place calls to the call center of a company at randomn times. There is only a single operator in this call center, and picks up the first call waiting when she is available. The customers are assumed to be extremely paint, as they wait as long as it gets to talk to the operator.

In [64]:
import simpy
import random
import numpy.random
from math import log,sqrt

Define a set of globals that define the characteristics of the model instance to be simulated. This includes the seed (RANDOM_SEED) for the random number generators, and key parameters for the interarrival (i.e. mean arrival rate) and service time (i.e. lower and upper bounds for the range) distribution.

•The inter-arrival times of calls are exponentially distributed with mean 14.3 min. 

•Incoming calls are first processed by the unskilled front-desk operator who records the personal details of the caller and the nature of the caller's request. When the operator is busy the customers are put on hold (they wait in a FCFS queue). 

•The time ittakes to collect and record the details of a caller is assumed to be LogNormal distributed with mean 7.2and standard deviation of 2.7minutes.

•Once this process is completed, the caller is directed to the expert operator, who tries to help  the  caller  with his/her  problem. As in the previous case, if the expert is busy the customers are put on hold (they wait in a FCFS queue). 

•Every customer that joins the queue leaves the queue without getting service following an independent exponentially distributed time with mean 60 minutes. (reneging).

•The service time of the expert is Exponentially distributed with mean 10.2minutes.

•The  expert  operator  takes  3-min  breaks  randomly  through  out  the  day.  When  the operator decides to take a break, he/she waits until completing all the customers already waiting  for  her/him.  If  new  customers  arrive  during  operators  break,  they  wait  in  the FCFS queue until the operator serves them. The operator resumes service after the break. 

•The  number  of  breaks  the  expert  operator  wishes to  take  during  an  8-hour  shift  is known to be distributed according to a Poisson distribution with a mean of 8 breaks per shift.

•Simulate this system for 1000 and 5000 answered calls separately.

•You should base your code on the SimPypseudocode provided in the jupyter notebook. 

•Collect and report statistics on: 

oUtilization of the front-desk operator

oUtilization of the expert operators, 

oAverage Total Waiting Time

oMaximumTotal Waiting Timeto Total System TimeRatio, and 

oAverage number of people waiting to be served by the expert operator.

In [67]:
# interarrival rates are exponentially distributed with mean 14.3 min
INTERARRIVAL_MEAN = 14.3 
# service time of first operator is LogNormal distributed with mean 7.2 and sd 2.7
m = 7.2
v = 2.7

FIRST_SERVICE_MEAN = log((m**2)/sqrt(v+m**2))
FIRST_SERVICE_STD = sqrt(log(v/(m**2)+1))


# customers leave queue if not serviced in exponentially distributed time with mean 60 min
RENEGE_TIME_MEAN = 60
# service time of expert is exponentially distributed with mean 10.2
EXPERT_SERVICE_MEAN = 10.2
# number of breaks taken by expert is Poisson distributed with mean 8 breaks
BREAKS_MEAN = 8
CALLNUM1=1000
CALLNUM2=5000

RANDOM_SEED = 978
random.seed(RANDOM_SEED)

Define the necessary set of arrays for bookkeeping

In [68]:
# service times for front-desk operator
first_service_times = [] 
# service times for expert 
second_service_times = []
# time spent by a customer while it waits for the operator (Queue waiting time Wq)
first_queue_w_times = [] 
second_queue_w_times = [] 

* The class definition for the customers arriving at the modeled system. When they are created, they immediatelly initiate a call (i.e. activate the call process). 

* Once a call is initiated, this is registered as a request to the operator resource. The customer is put on hold until the resource activates it back. 

* When the resource is available, the customer is activated and it then initiates the ask_question process. The duration of a question-answer session is determined randomly according to a uniform distribution.

In [75]:
class Customer(object):
    def __init__(self, name, env, operator,expert):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
    
    
    def call(self):
        print('%s initiated a call at %g' % (self.name, self.arrival_t))
 
        with operator.request() as op_req, expert.request() as exp_req :
            yield op_req
            print('%s is assigned to front-desk operator at %g' % (self.name, self.env.now))
            first_queue_w_times.append(self.env.now - self.arrival_t)
            yield self.env.process(self.ask_question_operator())
            self.finish_operator=self.env.now
            print('%s is done at %g at front-desk operator and directed to expert' % (self.name, self.finish_operator))
            
            
            self.patience=random.expovariate(1/RENEGE_TIME_MEAN)
            results = yield exp_req | env.timeout(self.patience)
            self.wait_exp=self.env.now-self.finish_operator
            first_queue_w_times.append(self.wait_exp) 
            
            if exp_req in results:
            # We got to the expert
                print('%7.4f %s: Waited %6.3f' % (self.env.now, self.name, self.wait_exp))
                yield self.env.process(self.ask_question_expert())
                self.finish_expert=self.env.now
                print('%s is done at %g at expert and exit the system' % (self.name, self.finish_expert))

            else:
            # We reneged
                print('%7.4f %s: RENEGED after %6.3f' % (self.finish_expert, self.name, self_wait_exp))
            

            
    def ask_question_operator(self):
        service_opr = random.lognormvariate(FIRST_SERVICE_MEAN,FIRST_SERVICE_STD)
        print("%s ask_question-op at  %g , service_opr= %g" % (self.name, self.env.now,service_opr))
        yield self.env.timeout(service_opr)
        print("%s ask_question-op at  %g finished " % (self.name, self.env.now))

        first_service_times.append(service_opr)


    def ask_question_expert(self):
        service_exp = random.expovariate(1/EXPERT_SERVICE_MEAN)
        yield self.env.timeout(service_exp)
        second_service_times.append(service_exp)
  
        


 
        

In [76]:
def customer_generator(env, operator,expert,callnum):
    """Generate new cars that arrive at the gas station."""
    for i in range(callnum):
        yield env.timeout(random.expovariate(1.0 / INTERARRIVAL_MEAN))
        customer = Customer('Cust %s' %(i+1), env, operator,expert)  

In [77]:
env = simpy.Environment()
callnum=CALLNUM1
operator = simpy.Resource(env, capacity = 1)
expert=simpy.Resource(env, capacity = 1)
env.process(customer_generator(env, operator,expert,callnum))
env.run() 

Cust 1 initiated a call at 20.0018
Cust 1 is assigned to front-desk operator at 20.0018
Cust 1 ask_question-op at  20.0018 , service_opr= 5.47286
Cust 2 initiated a call at 24.805
Cust 1 ask_question-op at  25.4747 finished 
Cust 1 is done at 25.4747 at front-desk operator and directed to expert
25.4747 Cust 1: Waited  0.000
Cust 1 is done at 45.2321 at expert and exit the system
Cust 2 is assigned to front-desk operator at 45.2321
Cust 2 ask_question-op at  45.2321 , service_opr= 7.84777
Cust 3 initiated a call at 47.2267
Cust 2 ask_question-op at  53.0799 finished 
Cust 2 is done at 53.0799 at front-desk operator and directed to expert
53.0799 Cust 2: Waited  0.000
Cust 2 is done at 60.2036 at expert and exit the system
Cust 3 is assigned to front-desk operator at 60.2036
Cust 3 ask_question-op at  60.2036 , service_opr= 7.10443
Cust 3 ask_question-op at  67.308 finished 
Cust 3 is done at 67.308 at front-desk operator and directed to expert
67.3080 Cust 3: Waited  0.000
Cust 3 is do

Cust 249 ask_question-op at  4257.84 , service_opr= 6.16836
Cust 249 ask_question-op at  4264.01 finished 
Cust 249 is done at 4264.01 at front-desk operator and directed to expert
4264.0133 Cust 249: Waited  0.000
Cust 279 initiated a call at 4264.41
Cust 249 is done at 4269.34 at expert and exit the system
Cust 250 is assigned to front-desk operator at 4269.34
Cust 250 ask_question-op at  4269.34 , service_opr= 9.90798
Cust 250 ask_question-op at  4279.25 finished 
Cust 250 is done at 4279.25 at front-desk operator and directed to expert
4279.2522 Cust 250: Waited  0.000
Cust 250 is done at 4297.08 at expert and exit the system
Cust 251 is assigned to front-desk operator at 4297.08
Cust 251 ask_question-op at  4297.08 , service_opr= 8.85406
Cust 280 initiated a call at 4301.34
Cust 281 initiated a call at 4304.01
Cust 251 ask_question-op at  4305.94 finished 
Cust 251 is done at 4305.94 at front-desk operator and directed to expert
4305.9355 Cust 251: Waited  0.000
Cust 251 is done a

Cust 534 initiated a call at 7810.78
Cust 452 is done at 7813.72 at expert and exit the system
Cust 453 is assigned to front-desk operator at 7813.72
Cust 453 ask_question-op at  7813.72 , service_opr= 4.52416
Cust 453 ask_question-op at  7818.24 finished 
Cust 453 is done at 7818.24 at front-desk operator and directed to expert
7818.2432 Cust 453: Waited  0.000
Cust 535 initiated a call at 7834.83
Cust 453 is done at 7846.02 at expert and exit the system
Cust 454 is assigned to front-desk operator at 7846.02
Cust 454 ask_question-op at  7846.02 , service_opr= 6.04424
Cust 454 ask_question-op at  7852.06 finished 
Cust 454 is done at 7852.06 at front-desk operator and directed to expert
7852.0625 Cust 454: Waited  0.000
Cust 454 is done at 7853.5 at expert and exit the system
Cust 455 is assigned to front-desk operator at 7853.5
Cust 455 ask_question-op at  7853.5 , service_opr= 6.49654
Cust 536 initiated a call at 7853.86
Cust 537 initiated a call at 7859.6
Cust 455 ask_question-op at

Cust 591 is done at 10426.4 at front-desk operator and directed to expert
10426.3884 Cust 591: Waited  0.000
Cust 591 is done at 10431.1 at expert and exit the system
Cust 592 is assigned to front-desk operator at 10431.1
Cust 592 ask_question-op at  10431.1 , service_opr= 8.62882
Cust 592 ask_question-op at  10439.7 finished 
Cust 592 is done at 10439.7 at front-desk operator and directed to expert
10439.6836 Cust 592: Waited  0.000
Cust 592 is done at 10443.1 at expert and exit the system
Cust 593 is assigned to front-desk operator at 10443.1
Cust 593 ask_question-op at  10443.1 , service_opr= 7.61087
Cust 593 ask_question-op at  10450.7 finished 
Cust 593 is done at 10450.7 at front-desk operator and directed to expert
10450.6804 Cust 593: Waited  0.000
Cust 724 initiated a call at 10455.2
Cust 593 is done at 10464.2 at expert and exit the system
Cust 594 is assigned to front-desk operator at 10464.2
Cust 594 ask_question-op at  10464.2 , service_opr= 7.55234
Cust 594 ask_question-o

Cust 801 is done at 14187.2 at front-desk operator and directed to expert
14187.1782 Cust 801: Waited  0.000
Cust 801 is done at 14188.3 at expert and exit the system
Cust 802 is assigned to front-desk operator at 14188.3
Cust 802 ask_question-op at  14188.3 , service_opr= 5.80657
Cust 964 initiated a call at 14188.5
Cust 802 ask_question-op at  14194.1 finished 
Cust 802 is done at 14194.1 at front-desk operator and directed to expert
14194.1207 Cust 802: Waited  0.000
Cust 802 is done at 14196.8 at expert and exit the system
Cust 803 is assigned to front-desk operator at 14196.8
Cust 803 ask_question-op at  14196.8 , service_opr= 7.74618
Cust 965 initiated a call at 14198.2
Cust 966 initiated a call at 14198.2
Cust 803 ask_question-op at  14204.6 finished 
Cust 803 is done at 14204.6 at front-desk operator and directed to expert
14204.5587 Cust 803: Waited  0.000
Cust 803 is done at 14208.5 at expert and exit the system
Cust 804 is assigned to front-desk operator at 14208.5
Cust 804 

In [43]:
print (first_queue_w_times)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2547.5002149500087, 0.0, 19606.14681366924, 0.0, 21089.290333427576, 0.0, 22135.676144580484, 0.0, 27683.600255443445, 0.0, 28469.042306071697, 0.0, 28510.394528353016, 0.0, 28606.658547840725, 0.0, 438267.574066151, 0.0, 441452.9792072548, 0.0, 441724.1540811806, 0.0, 878411.1340856599, 0.0, 879496.2516605956, 0.0, 988944.3716336807, 0.0, 989561.7270516712, 0.0, 989711.6654463178, 0.0, 999521.398768147, 0.0, 1009986.0426982406, 0.0, 1010055.7953852876, 0.0, 1010158.4862625436, 0.0, 1011496.8936014724, 0.0, 1011871.335682924, 0.0, 1011955.1719005317, 0.0, 1020956.1260660597, 0.0, 1021129.4134257737, 0.0, 1022690.4326883408, 0.0, 1023223.9481423224, 0.0, 1023410.728919172, 0.0, 1121824.2970803138, 0.0, 1125599.7721773111, 0.0, 1128573.1351315135, 0.0, 1159944.3610258747, 0.0, 1160133.2055305364, 0.0, 1263668.1184658015, 0.0, 1265318.7400124737, 0.0, 1265607.974202291, 0.0, 1268971.631019497, 0.0, 1277747.9393170492, 0.0, 1277749.6198694017, 0.0, 

In [123]:
print (second_queue_w_times)

[53.719422936944206, 74.36745758097598, 57.50398356266595, 86.20015959138377, 60.54594847560017, 64.54573727938265, 82.80127819535485, 79.15142152985119, 88.96147734612259, 80.14527961665469]
